<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">INFO</h1>

This file is a tutorial on how to use the PyRat library. \
It provides multiple examples of how to customize a game, add players, etc. \
This should give you an overview of how to create your own games.

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">IMPORTS</h1>

First, let's import everything we need in this tutorial.

The most important class is `Game`, that handles most things related to PyRat games. \
It will be in charge of creating a maze, placing cheese, asking players for decisions, rendering, etc.

Additionally, we will use the players defined in the `players` directory. \
By default, your workspace should contain four versions of a player playing at random. \
These are `Random1`, `Random2`, `Random3` and `Random4` below.

In [1]:
# External imports
import sys
import os
import pprint

# Add needed directories to the path
sys.path.append(os.path.join("..", "players"))

# PyRat imports
from pyrat import Game, PlayerSkin, StartingLocation, RenderMode, GameMode, Action, RandomMazeAlgorithm
from Random1 import Random1
from Random2 import Random2
from Random3 import Random3
from Random4 import Random4

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">CREATING A PYRAT GAME</h1>

To start a PyRat game, you just need to instantiate an object of the class `Game`. \
Then, add at least one player instance to the game (here, an instance of `Random4`) using method `add_player` of the `game` object. \
Finally, call the `start` method of the game object.

At this point, you should see a graphical window with the game inside. \
You can close it at any moment (use the cross, or escape key), this will not prevent the game from running.

Once the game is over, the `start` method will return a dictionary containing game statistics. \
We will detail this later in this tutorial.

In [ ]:
# Instantiate a game
game = Game()

# Instantiate a player and add it to the game
player = Random4()
game.add_player(player)

# Start the game
stats = game.start()

In a PyRat game, there can be either one player, as in the example we just did, or multiple players. \
In the latter case, players can be grouped in teams. \
The goal of a PyRat game is:
* If one team:
    * Catch all pieces of cheese.
    * The game will end when all pieces of cheese have been caught.
* If more than one team:
    * Catch more pieces of cheese than the other teams.
    * The game will end when no team can change its ranking.

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">CUSTOMIZING A PYRAT GAME</h1>

Let us customize the game by creating a match between two teams. \
To specify teams, we are going to precise the `team` argument in the `add_player` method.

By default, the game will distinguish the teams with distinct colors. \
For a better visualization, we can also give a distinct visual aspect to the players. \
To do so, we can specify the `skin` argument of a player, by giving it one value in the `PlayerSkin` enumeration.

Also, by default, players have the name of the class that define them (_e.g._, an instance of `Random1` will be named Random1). \
We can also customize this with the `name` argument of the player's constructor.

In [ ]:
# Instantiate a game
game = Game()

# Instantiate two players and add them to the game as a team named "Team 1"
# In Python, we can pass the arguments by name, so we can give them in any order
# This is done by specifying argument_name=argument_value when calling the function
player1 = Random1(skin=PlayerSkin.RAT, name="Rat")
player2 = Random2(skin=PlayerSkin.PYTHON, name="Python")
game.add_player(player1, team="Team 1")
game.add_player(player2, team="Team 1")

# Instantiate two players and add them to the game as another team named "Team 2"
# In Python, arguments can also be called without having to specity their names
# In this case, the order of the arguments is important, and should match the order of the arguments in the function definition
player3 = Random3("Ghost", PlayerSkin.GHOST)
player4 = Random4("Mario", PlayerSkin.MARIO)
game.add_player(player3, "Team 2")
game.add_player(player4, "Team 2")

# Start the game
stats = game.start()

As you may have noticed, players all start in the middle of the maze by default. \
This can be customized by setting the `location` argument of the `add_player` method. \
There are a few pre-defined locations in the `StartingLocation` enumeration. \
You can also specify the number of the cell where you want the player to start. \
If the asked location does not exist in the maze, the player will start at the closest existing location.

Let us create a game with a few players to illustrate starting player locations.

In [ ]:
# Instantiate a game
game = Game()

# Player 1 will start at the top left corner of the maze
# Note that since we do not specify the name of the team, we have to use named arguments, as the add_player function expects the team name as the second argument
player1 = Random4("TL")
game.add_player(player1, location=StartingLocation.TOP_LEFT)

# Player 2 will start at the top right corner of the maze
player2 = Random4("TR")
game.add_player(player2, location=StartingLocation.TOP_RIGHT)

# Player 3 will start at the bottom left corner of the maze
player3 = Random4("BL")
game.add_player(player3, location=StartingLocation.BOTTOM_LEFT)

# Player 4 will start at the bottom right corner of the maze
player4 = Random4("BR")
game.add_player(player4, location=StartingLocation.BOTTOM_RIGHT)

# Player 5 will start at the center of the maze
# Since this is the default location, it can also be omitted
player5 = Random4("C")
game.add_player(player5, location=StartingLocation.CENTER)

# Player 6 will start at a random location
player6 = Random4("R")
game.add_player(player6, location=StartingLocation.RANDOM)

# Player 7 will start at the same location as the previously added player
# I.e., it will start at the same location pas player 6
player7 = Random4("S")
game.add_player(player7, location=StartingLocation.SAME)

# Player 8 will start on cell 42
player8 = Random4("42")
game.add_player(player8, location=42)

# Start the game
stats = game.start()

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">CUSTOMIZING THE MAZE</h1>

As you have seen, by default, a PyRat game defines a maze with certain characteristics:
* The maze has a given width (in number of cells).
* The maze has a given height (in number of cells).
* Not all cells are accessible (i.e., the maze is not necessarily a rectangle and can contain holes).
* Some cells are separated with mud, indicating that more than one move is necessary to go from one to the other.
* There is a given number of walls.
* There are multiple pieces of cheese in the maze.

All these elements (and a few other ones) can be customized when instantiating the game. \
For instance, let us change the dimensions of the maze and the number of pieces of cheese.

In [ ]:
# Instatiate a game with specified arguments
game = Game(maze_width=10, maze_height=10, nb_cheese=1)

# Instantiate a player and add it to the game
player = Random4()
game.add_player(player)

# Start the game
stats = game.start()

Equivalently, it may be more practical to define the game configuration in a dictionary, and to use the dictionary contents as arguments of the game constructor.

In [ ]:
# Customize the game elements
config = {"maze_width": 10,
          "maze_height": 10,
          "nb_cheese": 1}

# Instatiate a game with specified arguments
game = Game(**config)

# Instantiate a player and add it to the game
player = Random4()
game.add_player(player)

# Start the game
stats = game.start()

So, what are the arguments we can set? \
Let us have a look at the definition of the constructor of class `Game`.

In [ ]:
# Print the documentation of the constructor of the Game class
print(Game.__init__.__doc__)

All these arguments can be set to customize a PyRat game at will. \
Here are the values they take by default, _i.e._, this is what defines a default PyRat game when no arguments are specified. \
These values are defined as class attributes of the `Game` class and all start with `DEFAULT`.

In [ ]:
# Show default values of the Game class constructor
for value in Game.__dict__:
    if value.startswith("DEFAULT"):
        print("Game.%s =" % value, Game.__dict__[value] if type(Game.__dict__[value]) != str else "\"%s\"" % Game.__dict__[value])

Some arguments can only take a particular value. \
This is the case for `render_mode`, that can only take a valued specified in the `RenderMode` enumeration. \
Here are the possible rendering modes.

In [ ]:
# Show the possible values of the RenderMode enumeration
print(RenderMode.__doc__)

This is also the case for the `game_mode`. \
Here are the possible values of the `GameMode` enumeration.

In [ ]:
# Show the possible values of the GameMode enumeration
print(GameMode.__doc__)

Finally, you can also choose the algorithm used to generate the random maze. \
Here are the possible values of the `RandomMazeAlgorithm` enumeration.

In [ ]:
# Show the possible values of the RandomMazeAlgorithm enumeration
print(RandomMazeAlgorithm.__doc__)

Not all arguments are compatible though. \
For instance, you cannot both specify a fixed list of pieces of cheese with `fixed_cheese`, and set `nb_cheese`, as the latter is used for random placement. \
If you specify an invalid configuration, you should get an error, either when instantiating the game, or when calling the `start` method.

In [ ]:
# Invalid configuration
config = {"fixed_cheese": [1, 2, 3],
          "nb_cheese": 3}

# Instatiate a game with specified arguments
game = Game(**config)

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">END OF GAME STATISTICS</h1>

As mentioned earlier, when a game is completed, the `start` method returns a dictionary of statistics that summarize what happened during the game. \
Let us have a look at its contents, by creating a game with multiple teams and players, using all we have seen above.

The `stats` dictionary contains the following entries:
* `stats["turns"]`: The number of turns of the game.
* `stats["players"]`: A dictionary for each player with name `"player_name"`, that contains the following entries:
    * `stats["players"]["player_name"]["actions"]`: A dictionary giving the number of times each action was chosen by the player, the number of turns it went into a wall, spend into mud, etc.
    * `stats["players"]["player_name"]["score"]`: The final score of the player.
    * `stats["players"]["player_name"]["preprocessing_duration"]`: The time spent in the `preprocessing` function of the player.
    * `stats["players"]["player_name"]["turn_durations"]`: A list of times spent in the `turn` function of the player.

In [ ]:
# Customize the game elements
config = {"maze_width": 10,
          "maze_height": 10,
          "mud_percentage": 30.0,
          "mud_range": (2, 7),
          "wall_percentage": 50.0,
          "cell_percentage": 90.0,
          "nb_cheese": 20}

# Instatiate a game with specified arguments
game = Game(**config)

# Instantiate and register players
# Here we make multiple teams of players, each team having a different type of player
# Team "Random 1" will start at the center of the maze (default)
team_1_name = "Random 1"
team_1_skin = PlayerSkin.RAT
for i in range(4):
    player_name = "P " + str(i + 1)
    player = Random1(player_name, team_1_skin)
    game.add_player(player, team_1_name)

# Team "Random 2" will start at the top left corner
# If such a cell does not exist, the players will start at the closest cell
team_2_name = "Random 2"
team_2_skin = PlayerSkin.PYTHON
team_2_start_location = StartingLocation.TOP_LEFT
for i in range(3):
    player_name = "P " + str(i + 5)
    player = Random2(player_name, team_2_skin)
    game.add_player(player, team_2_name, team_2_start_location)

# Team "Random 3" will start at a random location
# Location "same" indicates that the player will start at the same location as the previous player
team_3_name = "Random 3"
team_3_skin = PlayerSkin.GHOST
team_3_start_location = [StartingLocation.RANDOM, StartingLocation.SAME]
for i in range(2):
    player_name = "P " + str(i + 8)
    player = Random3(player_name, team_3_skin)
    game.add_player(player, team_3_name, team_3_start_location[i])

# Team "Random 4" will start at the the bottom right corner
# If such a cell does not exist, the player will start at the closest cell
team_4_name = "Random 4"
team_4_skin = PlayerSkin.MARIO
team_4_start_location = StartingLocation.BOTTOM_RIGHT
player = Random4("P 10", team_4_skin)
game.add_player(player, team_4_name, team_4_start_location)

# Start the game and show statistics when over
stats = game.start()
pprint.pprint(stats)

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">CREATING A CUSTOM PLAYER</h1>

There are very few things to do in order to create a player for a PyRat game. \
Here are the essential elements:
* A PyRat player should be a class that inherits from class `Player`.
* It should have a constructor (_i.e._, a method `__init__`) that will be executed when the class is instantiated.
* It should have a method `turn(self, maze, game_state)` that returns an action, as defined in the `Action` enumeration.
* Optionally, it can have a method `preprocessing(self, maze, game_state)`.
* Optionally, it can have a method `postprocessing(self, maze, game_state, stats)`.

Here are the possible actions that can be returned by the `turn` function.

In [ ]:
# Show the possible values of the Action enumeration
print(Action.__doc__)

The `Template.py` file provides a code that can be used as a basis for developing your own programs. \
Let us create a player, by adapting the template. \
This player plays randomly, and prints the history of moves it made when the game is over. \
Also, in the preprocessing, it prints the `maze` and `game_state` variables. \
Lines added to the template are identified with a `# NEW` comment.

In [2]:
#####################################################################################################################################################
###################################################################### IMPORTS ######################################################################
#####################################################################################################################################################

# External imports
from typing import *
from typing_extensions import *
from numbers import *
import random # NEW

# PyRat imports
from pyrat import Player, Maze, GameState, Action

#####################################################################################################################################################
###################################################################### CLASSES ######################################################################
#####################################################################################################################################################

class MyCustomPlayer (Player): # NEW

    """
        This player is basically a player that does nothing except printing the phase of the game.
        It is meant to be used as a template to create new players.
        Methods "preprocessing" and "postprocessing" are optional.
        Method "turn" is mandatory.
    """

    #############################################################################################################################################
    #                                                                CONSTRUCTOR                                                                #
    #############################################################################################################################################

    def __init__ ( self:     Self,
                   *args:    Any,
                   **kwargs: Any
                 ) ->        Self:

        """
            This function is the constructor of the class.
            In:
                * self:   Reference to the current object.
                * args:   Arguments to pass to the parent constructor.
                * kwargs: Keyword arguments to pass to the parent constructor.
            Out:
                * A new instance of the class.
        """

        # Inherit from parent class
        super().__init__(*args, **kwargs)

        # Initialize a list of moves
        self.moves = [] # NEW
       
    #############################################################################################################################################
    #                                                               PUBLIC METHODS                                                              #
    #############################################################################################################################################

    def preprocessing ( self:       Self,
                        maze:       Maze,
                        game_state: GameState,
                      ) ->          None:
        
        """
            This method redefines the method of the parent class.
            It is called once at the beginning of the game.
            In:
                * self:       Reference to the current object.
                * maze:       An object representing the maze in which the player plays.
                * game_state: An object representing the state of the game.
            Out:
                * None.
        """
        
        # Print the arguments received
        print(maze) # NEW
        print(game_state) # NEW

    #############################################################################################################################################

    def turn ( self:       Self,
               maze:       Maze,
               game_state: GameState,
             ) ->          Action:

        """
            This method redefines the abstract method of the parent class.
            It is called at each turn of the game.
            In:
                * self:       Reference to the current object.
                * maze:       An object representing the maze in which the player plays.
                * game_state: An object representing the state of the game.
            Out:
                * action: One of the possible actions.
        """

        # Select an action randomly, and store it
        action = random.choice(list(Action)) # NEW
        self.moves.append(action) # NEW

        # Return an action
        return action # NEW

#############################################################################################################################################

    def postprocessing ( self:       Self,
                         maze:       Maze,
                         game_state: GameState,
                         stats:      Dict[str, Any],
                       ) ->          None:

        """
            This method redefines the method of the parent class.
            It is called once at the end of the game.
            In:
                * self:       Reference to the current object.
                * maze:       An object representing the maze in which the player plays.
                * game_state: An object representing the state of the game.
                * stats:      Statistics about the game.
            Out:
                * None.
        """

        # Print the list of moves performed
        print(self.moves)

#####################################################################################################################################################
#####################################################################################################################################################

In order to use it in a game, just instantiate it and add it to the game, as before.

In [3]:
# Customize the game elements
config = {"maze_width": 5,
          "maze_height": 5,
          "nb_cheese": 1}

# Instatiate a game with specified arguments
game = Game(**config)

# Instantiate a player and add it to the game
player = MyCustomPlayer()
game.add_player(player)

# Start the game
stats = game.start()

Maze object:
|  Width: 5
|  Height: 5
|  Vertices: [2, 3, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
|  Adjacency matrix:
|  |  2 <-- (8) --> 3
|  |  2 <-- (1) --> 7
|  |  3 <-- (1) --> 8
|  |  6 <-- (1) --> 7
|  |  6 <-- (1) --> 11
|  |  7 <-- (1) --> 12
|  |  8 <-- (1) --> 9
|  |  8 <-- (1) --> 13
|  |  9 <-- (1) --> 14
|  |  11 <-- (9) --> 16
|  |  12 <-- (5) --> 13
|  |  12 <-- (1) --> 17
|  |  13 <-- (1) --> 14
|  |  13 <-- (1) --> 18
|  |  14 <-- (1) --> 19
|  |  15 <-- (1) --> 16
|  |  15 <-- (1) --> 20
|  |  16 <-- (1) --> 17
|  |  16 <-- (1) --> 21
|  |  17 <-- (1) --> 22
|  |  18 <-- (1) --> 23
|  |  19 <-- (8) --> 24
|  |  21 <-- (5) --> 22
GameState object:
|  Players: {'MyCustomPlayer': 12}
|  Scores: {'MyCustomPlayer': 0}
|  Muds: {'MyCustomPlayer': {'target': None, 'count': 0}}
|  Teams: {'': ['MyCustomPlayer']}
|  Cheese: [17]
|  Turn: 0

[<Action.NORTH: 'north'>, <Action.WEST: 'west'>, <Action.EAST: 'east'>, <Action.EAST: 'east'>, <Action.SOUTH

Have a look at the provided files in the `players` directory for multiple examples of PyRat players. \
Your players should be stored in Python files, just like provided examples, to use them easily from scripts that create games.